In [1]:
import os
os.chdir('/nfs/home/vaschetti/maxarSrc')
from pathlib import Path
from maxarseg.assemble import names, holders
from maxarseg.configs import Config
from deepforest import main
import torch
import rasterio
import numpy as np
from maxarseg.detect import detect, detect_utils
import geopandas as gpd
from maxarseg.samplers import samplers, samplers_utils

### Deep Forest

In [ ]:
torch.set_float32_matmul_precision('medium')
cfg = Config(config_path = Path('./configs/trees_cfg.yaml'))
tile_path = Path('/nfs/projects/overwatch/maxar-segmentation/maxar-open-data/Emilia-Romagna-Italy-flooding-may23/pre/10300100BF164000/120000301332.tif')

DF_model = main.deepforest(config_args = { 'devices' : cfg.get('models/df/device'),
                                        'retinanet': {'score_thresh': cfg.get('models/df/box_threshold')},
                                        'accelerator': 'cuda',
                                        'batch_size': cfg.get('models/df/bs')})
DF_model.use_release()

with rasterio.open(tile_path) as src:
    to_xy = src.xy
    crs = src.crs
    
for size in range(1000, 199, -100):
    cfg.set('models/df/size', size)
    print(cfg.get('models/df'))
    print(cfg.get('detection/trees/min_ratio_GD_boxes_edges'))
    boxes_df = DF_model.predict_tile(tile_path,
                                    return_plot = False,
                                    patch_size = cfg.get('models/df/size'),
                                    patch_overlap = cfg.get('models/df/patch_overlap'))
    boxes = boxes_df.iloc[:, :4].values
    score = boxes_df['score'].values
    
    glb_tile_tree_boxes = boxes
    glb_tile_tree_scores = score
    
    keep_ix_box_ratio = detect_utils.filter_on_box_ratio(glb_tile_tree_boxes,
                                                             min_edges_ratio = cfg.get('detection/trees/min_ratio_GD_boxes_edges'),
                                                             box_format = 'xyxy')
    glb_tile_tree_boxes = glb_tile_tree_boxes[keep_ix_box_ratio]
    glb_tile_tree_scores = glb_tile_tree_scores[keep_ix_box_ratio]
    print('box edge ratio filtering:', len(keep_ix_box_ratio) - np.sum(keep_ix_box_ratio), 'boxes removed')

    
    if len(glb_tile_tree_boxes.shape) == 1:
        glb_tile_tree_boxes = np.expand_dims(glb_tile_tree_boxes, axis = 0)
    if glb_tile_tree_scores.size == 1:
        glb_tile_tree_scores = np.expand_dims(glb_tile_tree_scores, axis = 0)
    
    for i, box in enumerate(glb_tile_tree_boxes):
        #need to invert x and y to go from col row to row col index
        try:
            glb_tile_tree_boxes[i] = np.array(to_xy(box[1], box[0]) + to_xy(box[3], box[2]))
        # catch and print 
        except Exception as e:
            print(f'Error in box {i}: {e}')                
    cols = {'score': list(glb_tile_tree_scores),
            'geometry': [samplers_utils.xyxyBox2Polygon(box) for box in glb_tile_tree_boxes]}

    gdf = gpd.GeoDataFrame(cols, crs = crs)
    gdf.to_file(tile_path.parts[-1][:-4]+'_tree' + str(size) +'_edg_ratio'+ str(cfg.get('detection/trees/min_ratio_GD_boxes_edges')) + '.gpkg', driver="GPKG")
    

## Dino

In [2]:
cfg = Config(config_path = Path('./configs/trees_cfg.yaml'))
print(cfg.get('models/gd'))

{'bs': 1, 'size': 600, 'stride': 400, 'device': 'cuda:0', 'root_path': './models/GDINO', 'config_file_path': './models/GDINO/configs/GroundingDINO_SwinT_OGC.py', 'weight_path': './models/GDINO/weights/groundingdino_swint_ogc.pth', 'text_prompt': 'bush', 'box_threshold': 0.12, 'text_threshold': 0.3}


In [3]:
events_names = names.get_all_events() 
event = holders.Event(events_names[1], cfg = cfg)

Evento su bordo
Creating event: Emilia-Romagna-Italy-flooding-may23
Region: Europe-Full
Mosaics: ['1040010030544E00', '1050410012C31100', '103005009DF96A00', '10300100C5C14E00', '104005003ADC5C00', '10300100C48E0A00', '10300500DDF89E00', '10300500CCF8B300', '10300100BF164000', '1050010032B2A600']


In [4]:
#GroundingDino
from groundingdino.util.inference import load_model as GD_load_model
from groundingdino.util.inference import predict as GD_predict
from groundingdino.util.inference import fast_predict as GD_fast_predict
from maxarseg.samplers import samplers, samplers_utils
from maxarseg.geo_datasets import geoDatasets
from torch.utils.data import DataLoader
from tqdm import tqdm
from rasterio.features import rasterize
from torchgeo.datasets import stack_samples

import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

tile_path = Path('/nfs/projects/overwatch/maxar-segmentation/maxar-open-data/Emilia-Romagna-Italy-flooding-may23/pre/10300100BF164000/120000301332.tif')

with rasterio.open(tile_path) as src:
    transform = src.transform
    tile_shape = (src.height, src.width)
    to_xy = src.xy
    crs = src.crs
gd_size = 1024
model = GD_load_model(cfg.get('models/gd/config_file_path'), cfg.get('models/gd/weight_path')).to(cfg.get('models/gd/device'))
print('\n- GD model device:', next(model.parameters()).device)

tile_aoi_gdf = samplers_utils.path_2_tile_aoi_no_water(tile_path, event.filtered_wlb_gdf)
aoi_mask = rasterize(tile_aoi_gdf.geometry, out_shape = tile_shape, fill=False, default_value=True, transform = transform)

dataset = geoDatasets.MxrSingleTileNoEmpty(str(tile_path), tile_aoi_gdf, aoi_mask=aoi_mask)
sampler = samplers.BatchGridGeoSampler(dataset, batch_size=cfg.get('models/gd/bs'), size=gd_size, stride=gd_size)
dataloader = DataLoader(dataset , batch_sampler=sampler, collate_fn=stack_samples, num_workers=4)

glb_tile_tree_boxes = torch.empty(0, 4)
all_logits = torch.empty(0)
model = model.to("cuda")
for batch in tqdm(dataloader, total = len(dataloader), desc="Detecting Trees with GDino"):
    img_b = batch['image'].permute(0,2,3,1).numpy().astype('uint8')
    
    for img, img_top_left_index in zip(img_b, batch['top_lft_index']):
        image_transformed = detect_utils.GD_img_load(img)
        tree_boxes, logits = GD_fast_predict(model,
                                            image_transformed,
                                            cfg.get('models/gd/text_prompt'),
                                            cfg.get('models/gd/box_threshold'),
                                            cfg.get('models/gd/text_threshold'),
                                            device = cfg.get('models/gd/device'))
        
        
        
        """rel_xyxy_tree_boxes = detect_utils.GDboxes2SamBoxes(tree_boxes, img_shape = gd_size)
        top_left_xy = np.array([img_top_left_index[1], #from an index to xyxy
                                img_top_left_index[0],
                                img_top_left_index[1],
                                img_top_left_index[0]])
        
        #turn boxes from patch xyxy coords to global xyxy coords
        glb_xyxy_tree_boxes = rel_xyxy_tree_boxes + top_left_xy
        
        glb_tile_tree_boxes = np.concatenate((glb_tile_tree_boxes, glb_xyxy_tree_boxes))
        all_logits = np.concatenate((all_logits, logits))"""

final text_encoder_type: bert-base-uncased

- GD model device: cuda:0
Completely contained in land. No mod to tile_aoi


Detecting Trees with GDino:  95%|█████████▌| 275/289 [02:22<00:06,  2.01it/s]

Discarded empty chips:  6
True num of batch:  283.0


Detecting Trees with GDino:  98%|█████████▊| 283/289 [02:26<00:03,  1.93it/s]


In [8]:
glb_tile_tree_scores = all_logits
copy_glb_tile_tree_boxes = glb_tile_tree_boxes.copy()
copy_glb_tile_tree_scores = glb_tile_tree_scores.copy()

In [9]:
keep_ix_box_area = detect_utils.filter_on_box_area_mt2(copy_glb_tile_tree_boxes,
                                                               max_area_mt2 = cfg.get('detection/trees/max_area_boxes_mt2'),
                                                               box_format = 'xyxy')
glb_tile_tree_boxes = copy_glb_tile_tree_boxes[keep_ix_box_area]
glb_tile_tree_scores = copy_glb_tile_tree_scores[keep_ix_box_area]
print('boxes area filtering: ', len(keep_ix_box_area) - np.sum(keep_ix_box_area), 'boxes removed')

keep_ix_box_ratio = detect_utils.filter_on_box_ratio(glb_tile_tree_boxes,
                                                        min_edges_ratio = cfg.get('detection/trees/min_ratio_GD_boxes_edges'),
                                                        box_format = 'xyxy')
glb_tile_tree_boxes = glb_tile_tree_boxes[keep_ix_box_ratio]
glb_tile_tree_scores = glb_tile_tree_scores[keep_ix_box_ratio]
print('box edge ratio filtering:', len(keep_ix_box_ratio) - np.sum(keep_ix_box_ratio), 'boxes removed')

boxes area filtering:  4231 boxes removed
box edge ratio filtering: 1077 boxes removed


In [10]:
cfg.get('detection/trees/max_area_boxes_mt2')

8000

In [11]:
if len(glb_tile_tree_boxes.shape) == 1:
            glb_tile_tree_boxes = np.expand_dims(glb_tile_tree_boxes, axis = 0)
if glb_tile_tree_scores.size == 1:
    glb_tile_tree_scores = np.expand_dims(glb_tile_tree_scores, axis = 0)

for i, box in enumerate(glb_tile_tree_boxes):
    #need to invert x and y to go from col row to row col index
    try:
        glb_tile_tree_boxes[i] = np.array(to_xy(box[1], box[0]) + to_xy(box[3], box[2]))
    # catch and print 
    except Exception as e:
        print(f'Error in box {i}: {e}')                
cols = {'score': list(glb_tile_tree_scores),
        'geometry': [samplers_utils.xyxyBox2Polygon(box) for box in glb_tile_tree_boxes]}

gdf = gpd.GeoDataFrame(cols, crs = crs)

gdf.to_file(tile_path.parts[-1][:-4]+'_Gdino'+'_tree' + str(gd_size) +'_edg_ratio'+ str(cfg.get('detection/trees/min_ratio_GD_boxes_edges')) + '_area'+str(cfg.get('detection/trees/max_area_boxes_mt2'))+'.gpkg', driver="GPKG")